In [ ]:
import json
import glob
import re
import os
import xlsxwriter
import html2text

In [206]:
#폴더 경로
folder_name = "./tagtog_relation_extraction/RE-Data/"

#context list
context_name_list = os.listdir(folder_name + "ann.json/master/pool")

#relation 폴더 경로
relation_folder_paths = glob.glob(folder_name + "ann.json/master/pool/*")

#context 폴더 경로
# contexts_folders_paths = glob.glob(folder_name + "plain.html/pool/*")
contexts_folders_paths = [folder_name + "plain.html/pool/" + c for c in context_name_list]


#anntation_lenged 정보
annotation_legend = folder_name + "annotations-legend.json"
with open(annotation_legend,"r") as f:
    annotation_legend = json.load(f)

In [209]:
#html 파일로부터 text만 추출해주는 함수
def get_context_from_html(html_file):
    html_file = re.sub(r"\n","", html_file)
    return re.findall("(<pre.+>)(.+)(</pre>)",html_file)[0][1]

In [210]:
def return_entity(example):
  result_dict = {}
  entity = annotation_legend[example["classId"]].split("-")
  result_dict['entity'] = entity[0]
  result_dict['type'] = entity[1]
  result_dict['text'] = example["offsets"][0]["text"]
  result_dict['index'] = example["offsets"][0]["start"]
  return result_dict

In [211]:
h = html2text.HTML2Text()
workbook = xlsxwriter.Workbook('demo.xlsx')
  

In [212]:
worksheet = workbook.add_worksheet()

red = workbook.add_format({'color': 'red'})
blue = workbook.add_format({'color': 'blue'})
text_wrap = workbook.add_format({'text_wrap': True})
worksheet.set_column('I:I', 100)
worksheet.set_column('G:G', 15)
worksheet.set_column('H:H', 15)

worksheet.write("A1", 'id')
worksheet.write("B1", 'sub_entity')
worksheet.write("C1", 'obj_entity')
worksheet.write("D1", 'sub_index')
worksheet.write("E1", 'obj_index')
worksheet.write("F1", 'relation')
worksheet.write("G1", 'sub_text')
worksheet.write("H1", 'obj_text')
worksheet.write("I1", 'sentence')

index = 2

for json_dir in relation_folder_paths:
  html_dir = os.path.join(folder_name,"plain.html/pool", json_dir[-2:])
  file_list = os.listdir(json_dir)
  for json_file_name in file_list:
    with open(os.path.join(json_dir, json_file_name), "r") as f:
      json_file = json.load(f)
    with open(os.path.join(html_dir, json_file_name[:-8] + 'plain.html'), "r") as f:
      text = get_context_from_html(f.read())
    text = h.handle(text)
    text = re.sub(r"\n"," ", text)
    entities = {}
    for entity in json_file["entities"]:
      entities[f"{entity['part']}|{entity['classId']}|{entity['offsets'][0]['start']},{entity['offsets'][0]['start'] + len(entity['offsets'][0]['text']) - 1}"] = return_entity(entity)
    for relation in json_file["relations"]:
      id = f"{json_dir[-2:]}_{int(json_file_name[-17:-13]):05d}"
      entity1, entity2 = entities[relation["entities"][0]], entities[relation["entities"][1]]
      sub_entity, obj_entity = (entity1, entity2) if entity1['entity'] == "SUBJ" else (entity2, entity1)
      if sub_entity['index'] > obj_entity['index']:
        colored_text = [text[:obj_entity['index']], blue, obj_entity['text'], text[obj_entity['index'] + len(obj_entity['text']):sub_entity['index']], red, sub_entity['text'], text[sub_entity['index'] + len(sub_entity['text']):]]
      else:
        colored_text = [text[:sub_entity['index']], red, sub_entity['text'], text[sub_entity['index'] + len(sub_entity['text']):obj_entity['index']], blue, obj_entity['text'], text[obj_entity['index'] + len(obj_entity['text']):]]
      colored_text = list(filter(lambda x: x != '', colored_text))
      colored_text.append(text_wrap)
      worksheet.write(f"A{index}", id)
      worksheet.write(f"B{index}", sub_entity['entity'])
      worksheet.write(f"C{index}", obj_entity['entity'])
      worksheet.write(f"D{index}", sub_entity['index'])
      worksheet.write(f"E{index}", obj_entity['index'])
      worksheet.write(f"F{index}", annotation_legend[relation['classId']])
      worksheet.write(f"G{index}", sub_entity['text'])
      worksheet.write(f"H{index}", obj_entity['text'])
      worksheet.write_rich_string(f"I{index}", *colored_text)
      index += 1
workbook.close()

In [213]:
def return_entity(example):
  result_dict = {}
  entity = annotation_legend[example["classId"]].split("-")
  result_dict['entity'] = entity[0]
  result_dict['type'] = entity[1]
  result_dict['text'] = example["offsets"][0]["text"]
  result_dict['index'] = example["offsets"][0]["start"]
  return result_dict